#### Label Encoding
-단순 수치값으로 mapping하는 작업

### 원핫인코딩
-Label 인코딩 or Ont-hot 인코딩 방식을 이용해 수치화

### 목표
- 버섯의 특징(특성)을 활용해서 독/식용 버섯분류
- Deicision Tree 모델 시각화 & 과대적합 제어
- 특성중요도를 뽑아 보자 : 특성 선택할 수 있는 방법 중 하나   
( 최빈값 == 가장많이 관측되는 값 )

In [1]:
#판다스
#랜덤 샘플링, 7:3
#tree // from skearn free import DecisionTreeClassifier

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics

1. 버섯 데이터로드 ( data/mushroom.csv )
2. 전체 컬럼, 행 크기파악
3. 결측치 확인
4. 기술통계 -> 범주형데이터 : 개수, 최빈값 등..
5. 문제와 답 X, y
6. 정답 label 안에 독버섯과 식용 버섯의 비율 확인
7. 범주형 -> 수치화(원핫 인코딩, 레이블인코딩 )
8. train.test 분리
9. 모델링

In [2]:
#버섯 데이터 불러와 주세요
data=pd.read_csv('data/mushroom.csv')
data.head()

poisonous cap-shape cap-surface cap-color bruises odor gill-attachment  \
0         p         x           s         n       t    p               f   
1         e         x           s         y       t    a               f   
2         e         b           s         w       t    l               f   
3         p         x           y         w       t    p               f   
4         e         x           s         g       f    n               f   

  gill-spacing gill-size gill-color  ... stalk-surface-below-ring  \
0            c         n          k  ...                        s   
1            c         b          k  ...                        s   
2            c         b          n  ...                        s   
3            c         n          n  ...                        s   
4            w         b          k  ...                        s   

  stalk-color-above-ring stalk-color-below-ring veil-type veil-color  \
0                      w                      w         p          w   
1                      w                      w         p          w   
2                      w                      w         p          w   
3                      w                      w         p          w   
4                      w                      w         p          w   

  ring-number ring-type spore-print-color population habitat  
0           o         p                 k          s       u  
1           o         p                 n          n       g  
2           o         p                 n          n       m  
3           o         p                 k          s       u  
4           o         e                 n          a       g  

[5 rows x 23 columns]

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8124 entries, 0 to 8123
Data columns (total 23 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   poisonous                 8124 non-null   object
 1   cap-shape                 8124 non-null   object
 2   cap-surface               8124 non-null   object
 3   cap-color                 8124 non-null   object
 4   bruises                   8124 non-null   object
 5   odor                      8124 non-null   object
 6   gill-attachment           8124 non-null   object
 7   gill-spacing              8124 non-null   object
 8   gill-size                 8124 non-null   object
 9   gill-color                8124 non-null   object
 10  stalk-shape               8124 non-null   object
 11  stalk-root                8124 non-null   object
 12  stalk-surface-above-ring  8124 non-null   object
 13  stalk-surface-below-ring  8124 non-null   object
 14  stalk-color-above-ring  

In [4]:
data.describe() #기술통계 확인 freq 최빈값(빈도수가 가장 높은 , 많은)

poisonous cap-shape cap-surface cap-color bruises  odor  \
count       8124      8124        8124      8124    8124  8124   
unique         2         6           4        10       2     9   
top            e         x           y         n       f     n   
freq        4208      3656        3244      2284    4748  3528   

       gill-attachment gill-spacing gill-size gill-color  ...  \
count             8124         8124      8124       8124  ...   
unique               2            2         2         12  ...   
top                  f            c         b          b  ...   
freq              7914         6812      5612       1728  ...   

       stalk-surface-below-ring stalk-color-above-ring stalk-color-below-ring  \
count                      8124                   8124                   8124   
unique                        4                      9                      9   
top                           s                      w                      w   
freq                       4936                   4464                   4384   

       veil-type veil-color ring-number ring-type spore-print-color  \
count       8124       8124        8124      8124              8124   
unique         1          4           3         5                 9   
top            p          w           o         p                 w   
freq        8124       7924        7488      3968              2388   

       population habitat  
count        8124    8124  
unique          6       7  
top             v       d  
freq         4040    3148  

[4 rows x 23 columns]

### X, y 문제, 답 분리

In [5]:
#전체데이터 data 문제 인덱싱
# poisonous를 제외한 값
X=data.iloc[:, 1:]
X.shape

(8124, 22)

In [6]:
#전체 데이터 data 답 인덱싱
# poisonous 값
y=data['poisonous']
y.shape

(8124,)

In [7]:
#label정답의 비율을 한번 살펴보자
#value_counts()
#e => 식용버섯, p => 독버섯
y.value_counts()

e    4208
p    3916
Name: poisonous, dtype: int64

### 원핫 인코딩 : 범주형(이산형) 특성을 수치화

In [8]:
X_one_hot=pd.get_dummies(X)
X_one_hot.head()

cap-shape_b  cap-shape_c  cap-shape_f  cap-shape_k  cap-shape_s  \
0            0            0            0            0            0   
1            0            0            0            0            0   
2            1            0            0            0            0   
3            0            0            0            0            0   
4            0            0            0            0            0   

   cap-shape_x  cap-surface_f  cap-surface_g  cap-surface_s  cap-surface_y  \
0            1              0              0              1              0   
1            1              0              0              1              0   
2            0              0              0              1              0   
3            1              0              0              0              1   
4            1              0              0              1              0   

   ...  population_s  population_v  population_y  habitat_d  habitat_g  \
0  ...             1             0             0          0          0   
1  ...             0             0             0          0          1   
2  ...             0             0             0          0          0   
3  ...             1             0             0          0          0   
4  ...             0             0             0          0          1   

   habitat_l  habitat_m  habitat_p  habitat_u  habitat_w  
0          0          0          0          1          0  
1          0          0          0          0          0  
2          0          1          0          0          0  
3          0          0          0          1          0  
4          0          0          0          0          0  

[5 rows x 117 columns]

In [9]:
#label encoding 
X['habitat'].unique()

array(['u', 'g', 'm', 'd', 'p', 'w', 'l'], dtype=object)

In [10]:
habitat_dic={
    'u':2,
    'g':1,
    'm':3,
    'd':4,
    'p':5,
    'w':7,
    'l':6
}

In [11]:
X['habitat'].map(habitat_dic)

0       2
1       1
2       3
3       2
4       1
       ..
8119    6
8120    6
8121    6
8122    6
8123    6
Name: habitat, Length: 8124, dtype: int64

- 원핫 인코딩   
 랭킹값이 아닐경우에는 즉 순서가 없을경우는 궂이 label encoding할 필요가 없음    
 각각의 고유값 

- 레이블 인코딩   
 순서의 의미가 있을 때(예, 유치원->초등->중등->고등->대딩   
 평사원->대리->과장 등의 순차)

### 모델링
X_train, X_test, y_train, y_test 로 분할

In [12]:
X_train, X_test, y_train, y_test=train_test_split(X_one_hot, y, test_size=0.3, random_state=65)

In [13]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(5686, 117)
(2438, 117)
(5686,)
(2438,)


### 모델객체 생성, 하이퍼파라미터, 학습, 예측, 정확도

In [14]:
tree_model=DecisionTreeClassifier(max_depth=5)

In [15]:
tree_model.fit(X_train, y_train) #학습하기

DecisionTreeClassifier(max_depth=5)

In [16]:
#예측하기
pre = tree_model.predict(X_test)

In [17]:
from sklearn import metrics

In [18]:
print(y_test.shape)
print(pre.shape)

(2438,)
(2438,)


In [19]:
metrics.accuracy_score(y_test, pre)

0.9995898277276456

In [20]:
# 실행이 제대로 안되었나봐요끝! 몇번을 눌러줘는데도 그러는군요 허허
#저도 하다가 가끔 화가납니다..ㅎㅎ 

In [21]:
#train 정확도도 한번 뽑아보자
pre=tree_model.predict(X_train)
# 승원씨 마크다운 없애는 방법은 y를 눌러주면 됩니다 !~ ㅋㅋ

In [22]:
metrics.accuracy_score(y_train, pre)

0.9996482588814632

### 과대 적합 제어
만약 과대적합일 확률이 높은 경우, 그 모델은 일반화 되지 않은 모델이라고 판단  
일반화를 시켜주기 위해서 사용

In [23]:
#tree 모델 생성, max_depth=1
tree_model2=DecisionTreeClassifier(max_depth=1)

In [24]:
#fit
tree_model2.fit(X_train, y_train) #학습하기

DecisionTreeClassifier(max_depth=1)

In [25]:
#예측
pre2=tree_model2.predict(X_test)

In [26]:
#정확도 확인
metrics.accuracy_score(pre2, y_test)
#accuracy_score(예측값, test정답)

0.8859721082854799

In [27]:
#정확도 확인2
pre_train=tree_model2.predict(X_train)

In [28]:
metrics.accuracy_score(pre_train, y_train)

0.887091100949701

In [29]:
#모델 자체에서 제공해주는 정확도(test에 대한 정답이 있을 경우)
tree_model2.score(X_test, y_test)

0.8859721082854799

### 특성선택
-tree모델의 특성 중요도

In [30]:
fi=tree_model.feature_importances_

In [31]:
import_df=pd.DataFrame(fi, index=X_train.columns)
import_df.sort_values(by=0, ascending=False).head()

0
odor_n               0.618496
stalk-root_c         0.179217
stalk-root_r         0.082643
spore-print-color_r  0.033528
odor_l               0.023113